In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from taskdataset import TaskDataset
import matplotlib.pyplot as plt
import os
import time
import pickle as pkl

from t2_functions import partition_ids, partition_ids2

In [2]:
import sys

sys.path.append(os.path.join(os.getcwd(), "task_2_sybilattack/"))

from endpoints.requests import sybil, sybil_reset


In [3]:
print(os.getcwd())
dataset = torch.load("data/SybilAttack.pt")

In [4]:
ids = np.array(dataset.ids)
img_vecs = dataset.imgs
print(np.array(img_vecs[0]).size)
bins = partition_ids2(ids, img_vecs, main_bin_num=10)


1024


  1%|          | 200/18092 [00:01<02:20, 127.21it/s]


num: 200


  1%|          | 200/17892 [00:01<02:12, 133.59it/s]


num: 200


...

num: 200


  1%|          | 200/16492 [00:01<02:00, 135.35it/s]


num: 200


  1%|          | 200/16292 [00:01<02:28, 108.18it/s]

num: 200


In [ ]:
task = 'affine'
7=4
sybil_reset(home_or_defense='home', binary_or_affine=task)
sybil_reset(home_or_defense='defense', binary_or_affine=task)


Request ok
{'msg': 'Successful sybil affine reset home'}
Request ok
{'msg': 'Successful sybil affine reset defense'}


In [125]:
ids_train, ids_test = bins[0]
print(len(ids_train))
print(len(ids_test))
A_train_reps = sybil(ids=ids_train,
                 home_or_defense='home',
                 binary_or_affine=task)

B_train_reps = sybil(ids=ids_train,
                 home_or_defense='defense',
                 binary_or_affine=task)
print(f"A train reps: {len(A_train_reps)}")

A_test_reps = sybil(ids=ids_test,
                 home_or_defense='home',
                 binary_or_affine=task)

B_test_reps = sybil(ids=ids_test,
                 home_or_defense='defense',
                 binary_or_affine=task)
print(f"A test reps: {len(A_test_reps)}")


200
1800
A train reps: 200
A test reps: 1800


In [126]:
task='affine_v3'
os.makedirs(f'data/{task}', exist_ok=True)

with open(f'data/{task}/A_train', 'wb') as f:
    pkl.dump(A_train_reps, f)
with open(f'data/{task}/B_train', 'wb') as f:
    pkl.dump(B_train_reps, f)
with open(f'data/{task}/A_test', 'wb') as f:
    pkl.dump(A_test_reps, f)
with open(f'data/{task}/B_test', 'wb') as f:
    pkl.dump(B_test_reps, f)


In [139]:
task='affine_v3'

with open(f'data/{task}/A_train', 'rb') as f:
    A_train_reps = pkl.load(f)
with open(f'data/{task}/B_train', 'rb') as f:
    B_train_reps = pkl.load(f)
with open(f'data/{task}/A_test', 'rb') as f:
    A_test_reps = pkl.load(f)
with open(f'data/{task}/B_test', 'rb') as f:
    B_test_reps = pkl.load(f)
    
print(f"A train reps: {len(A_train_reps)}")
print(f"A test reps: {len(A_test_reps)}")


A train reps: 200
A test reps: 1800


In [140]:
class RepresentationsDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]
        return torch.tensor(x), torch.tensor(y)


In [141]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [142]:
class Linear(nn.Module):
    def __init__(self, input_size, output_size):
        super(Linear, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
    
    api_bina_t = torch.tensor(api_bina,dtype=torch.float32)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_MSE = nn.MSELoss()

    # Training loop
    num_epochs = 60
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader):
            model.train()
            inputs, labels = data
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss_tr = loss_MSE(outputs, labels) #+ l1_penalty(model, 1e-7)
            
            # Backward pass and optimize
            loss_tr.backward()
            optimizer.step()
            
        
        """
        if (epoch+1) % 10 == 0:  # Print every 10 epoch
            model.eval()

            with torch.inference_mode():
                out = model(bin_sybil_def_tensor)
                MSE = loss_MSE(out,bin_sybil_home_tensor)

                out_t = model(X_test_sm_t)
                MSE_t = loss_MSE(out_t,y_test_sm_t)
            print(f"Epoch [{epoch+1}/{num_epochs}], loss whole: {MSE}, loss test: {MSE_t}, train_loss : {loss_tr}]")
        """
    out = model(api_bina_t)
    if len(inds)==0:
        inds = index
    else:
        inds = np.concatenate((inds,index))

    if len(repr_home) == 0:
        repr_home = api_bin_home
    else:
        repr_home = np.concatenate((repr_home,api_bin_home))

    if len(repr_def) == 0:
        repr_def = api_bina
    else:
        repr_def = np.concatenate((repr_def,api_bina))
    
    if len(represen) == 0:
        represen = out.detach().numpy()
    else:
        represen = np.concatenate((represen,out.detach().numpy()))
        

In [19]:
print(len(inds))
print(represen.shape)
print(repr_home.shape)
print(repr_def.shape)
print(list(range(1,10)))

6000
(6000, 384)
(600, 384)
(6000, 384)
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [18]:
for i in range(1,10):
    print(f"epoch: {i}")
    index = dataset.ids[i*2000:(i+1)*2000]
    sybil_reset(binary_or_affine = 'binary', home_or_defense ='defense')
    index_home = index[:200]

    time.sleep(10)
    api_bina = np.array(sybil(ids=index,
                    home_or_defense='defense',
                    binary_or_affine='binary'))
    api_bin_home = np.array(sybil(ids=index_home,
                    home_or_defense='home',
                    binary_or_affine='binary'))
    

    model = MLP(384,600,0,384)
    X_train_sm, y_train_sm = api_bina[:200], api_bin_home[:200]
    #X_test_sm, y_test_sm = bin_sybil_def[200:], bin_sybil_home[200:]

    '''
    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    '''
    #X_test_sm_t, y_test_sm_t = torch.tensor(X_test_sm, dtype=torch.float32),torch.tensor(y_test_sm, dtype=torch.float32)


    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_sm, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_sm, dtype=torch.float32)

    #bin_sybil_def_tensor =torch.tensor(bin_sybil_def, dtype=torch.float32)
    #bin_sybil_home_tensor =torch.tensor(bin_sybil_home, dtype=torch.float32)

    #print(X_train.shape)
    #print(X_train_tensor.shape)

    # Create DataLoader for batch training
    #train_dataset = TensorDataset(X_train_tensor.reshape((384,1600)), y_train_tensor.reshape((384,1600)))
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size =32, shuffle=True)
    
    api_bina_t = torch.tensor(api_bina,dtype=torch.float32)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_MSE = nn.MSELoss()

    # Training loop
    num_epochs = 60
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader):
            model.train()
            inputs, labels = data
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss_tr = loss_MSE(outputs, labels) #+ l1_penalty(model, 1e-7)
            
            # Backward pass and optimize
            loss_tr.backward()
            optimizer.step()
            
        
        """
        if (epoch+1) % 10 == 0:  # Print every 10 epoch
            model.eval()

            with torch.inference_mode():
                out = model(bin_sybil_def_tensor)
                MSE = loss_MSE(out,bin_sybil_home_tensor)

                out_t = model(X_test_sm_t)
                MSE_t = loss_MSE(out_t,y_test_sm_t)
            print(f"Epoch [{epoch+1}/{num_epochs}], loss whole: {MSE}, loss test: {MSE_t}, train_loss : {loss_tr}]")
        """
    out = model(api_bina_t)
    if len(inds)==0:
        inds = index
    else:
        inds = np.concatenate((inds,index))

    if len(repr_home) == 0:
        repr_home = api_bin_home
    else:
        repr_home = np.concatenate((repr_home,api_bin_home))

    if len(repr_def) == 0:
        repr_def = api_bina
    else:
        repr_def = np.concatenate((repr_def,api_bina))

    if len(represen) == 0:
        represen = out.detach().numpy()
    else:
        represen = np.concatenate((represen,out.detach().numpy()))

    np.savez(
        f"task_2_sybilattack/data/my_submission_{i}.npz",
        ids=inds,
        representations=represen,
        repr_def = repr_def,
        repr_home = repr_home
    )
        

epoch: 1


ConnectionError: HTTPConnectionPool(host='34.71.138.79', port=9090): Max retries exceeded with url: /sybil/binary/reset/defense (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000016830F4E810>: Failed to establish a new connection: [WinError 10061] Nie można nawiązać połączenia, ponieważ komputer docelowy aktywnie go odmawia'))

In [24]:
np.savez(
    "./data/my_submission_2.npz",
    ids=inds,
    representations=represen,
    repr_def = repr_def,
    repr_home = repr_home
)

In [ ]:
np.savez(
        "task_2_sybilattack/data/my_submission.npz",
        ids=inds,
        representations=represen,
    )

torch.float64

In [143]:
def validate(criterion, loader, net):
    net.eval()

    with torch.no_grad():
        true = []
        pred = []
        for x, y in loader:
            y_hat = net(x)
            true.append(y.numpy())
            pred.append(y_hat.numpy())

        true = torch.tensor(np.concatenate(true, axis=0))
        pred = torch.tensor(np.concatenate(pred, axis=0))

        loss = criterion(pred, true)

    return loss


In [144]:
def l1_reg(model, reg_lambda):
    l1_regularization = torch.tensor(0., device=model.parameters().__next__().device)
    for param in model.parameters():
        l1_regularization += torch.norm(param, p=1)
    return reg_lambda * l1_regularization


In [145]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train(epochs, optim, criterion, regularise, trainloader, valloader, net, empty_net, reg_lambda=None):
    best_val_loss = np.inf
    best_net = empty_net

    for epoch in range(epochs):  # loop over the dataset multiple times
        train_loss = 0

        for iter, (x, y) in enumerate(tqdm(trainloader)):
            net.train()
            optim.zero_grad()

            y_hat = net(x)
            loss = criterion(y_hat, y)
            if reg_lambda is not None:
                loss += regularise(net, reg_lambda)
            loss.backward()
            optim.step()

            batch_loss = loss.item()
            train_loss += batch_loss

        train_loss /= len(trainloader)
        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}")

        net.eval()

        val_loss = validate(criterion, valloader, net)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_net.load_state_dict(net.state_dict())

        print(f"Epoch [{epoch + 1}/{epochs}], Val Loss: {val_loss:.4f}")

    return net, best_net


In [146]:
criterion = nn.MSELoss()
batch_size = 32
epochs = 10
lr = 0.001


In [171]:
sybil_reset(home_or_defense='defense', binary_or_affine='affine')
time.sleep(10)
ids = [69144, 171315, 769, 178092, 158444, 158744, 172621, 167283, 72307, 164047, 258664, 3590, 284093, 41717, 3567, 159343, 64894, 159966, 164186, 164071, 163258, 301881, 165550, 172130, 69389, 165760, 11591, 170440, 186115, 121592, 2867, 16164, 156724, 973, 37599, 163039, 59390, 165616, 2633, 159790, 160140, 94940, 164504, 5940, 104320, 155026, 167807, 88868, 33587, 293377, 162958, 160560, 172363, 67047, 100569, 161985, 199726, 218587, 21247, 294393, 1834, 164320, 103489, 278419, 169492, 107264, 55625, 94323, 160807, 168318, 166308, 164923, 280775, 170576, 66131, 161881, 263079, 1175, 89046, 246133, 160413, 164142, 249396, 33044, 67711, 153291, 266561, 3157, 855, 221, 6, 15106, 42508, 48457, 161555, 168330, 249443, 159412, 2030, 25128, 136081, 1783, 172381, 19782, 50452, 160756, 793, 167057, 167301, 3016, 45241, 51900, 47227, 172289, 292065, 74241, 299354, 157148, 186303, 94542, 166308, 144989, 144122, 3490, 14845, 172043, 187, 127550, 170021, 1159, 170068, 157090, 21335, 155662, 191240, 182295, 39092, 203608, 67515, 121906, 169314, 251808, 33934, 136784, 170118, 81612, 162028, 229049, 32472, 165710, 7812, 163670, 228941, 168505, 235070, 84322, 179778, 163875, 260037, 230, 7418, 257761, 2907, 160548, 258990, 53649, 172221, 302226, 163400, 877, 227397, 184456, 261492, 185136, 163924, 70048, 154093, 48548, 99050, 3644, 245419, 144379, 202640, 15181, 234344, 3777, 3161, 172284, 127623, 466, 297564, 13788, 76386, 43239, 202394, 3969, 168062, 164856, 122274, 52723]

sybil(ids, home_or_defense='defense', binary_or_affine='affine')


Request ok
{'msg': 'Successful sybil affine reset defense'}


[[0.1599130630493164,
  -0.39405807852745056,
  -0.4857172966003418,
  1.708618402481079,
  0.5116194486618042,
  -0.13342715799808502,
  0.5474154949188232,
  -0.17326366901397705,
  -1.083664894104004,
  -0.05171215534210205,
  -0.12384525686502457,
  -2.1927175521850586,
  0.6267749667167664,
  -3.883448362350464,
  2.2369205951690674,
  -1.5713911056518555,
  -3.737396240234375,
  -0.1306636929512024,
  0.17467670142650604,
  0.15295077860355377,
  1.9270594120025635,
  -1.4836384057998657,
  -0.9858924150466919,
  -0.6921612620353699,
  -0.21744635701179504,
  2.429678440093994,
  -0.784825325012207,
  -1.9127806425094604,
  0.8594112396240234,
  1.201326847076416,
  -0.8738930225372314,
  4.9654669761657715,
  0.13252578675746918,
  0.24357320368289948,
  -1.2920067310333252,
  0.5776649117469788,
  -0.1450260430574417,
  1.7101941108703613,
  2.70863676071167,
  0.9926256537437439,
  4.182811260223389,
  0.12447988986968994,
  -0.5429465770721436,
  -0.7846792936325073,
  -1.161

In [147]:
def lin_augment_affine(xs, ys, new_for_each_pair=3):
    new_x = []
    new_y = []
    for i, (x1, y1) in tqdm(enumerate(zip(xs, ys))):
        for j, (x2, y2) in enumerate(zip(xs, ys)):
            if i != j:
                weights = np.random.uniform(-1, 2, size=new_for_each_pair)
                for w in weights:
                    x_new = w*x1 + (1-w)*x2
                    y_new = w*y1 + (1-w)*y2
                    new_x.append(x_new)
                    new_y.append(y_new)
    new_x = np.array(new_x)
    new_y = np.array(new_y)
    return np.concatenate((xs, new_x), axis=0), np.concatenate((ys, new_y), axis=0)


In [155]:
A_train_aug = torch.tensor(A_train_reps, dtype=torch.float32)
B_train_aug = torch.tensor(B_train_reps, dtype=torch.float32)
A_train_aug, B_train_aug = lin_augment_affine(A_train_aug, B_train_aug)

A_test_reps = torch.tensor(A_test_reps, dtype=torch.float32)
B_test_reps = torch.tensor(B_test_reps, dtype=torch.float32)

200it [00:06, 29.80it/s]


In [156]:
train_dataset = RepresentationsDataset(x=A_train_aug, y=B_train_aug)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = RepresentationsDataset(x=A_test_reps, y=B_test_reps)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [157]:
lin_net = Linear(384, 384)
lin_empty_net = Linear(384, 384)

optim = torch.optim.Adam(lr=lr, params=lin_net.parameters())

lin_last_net, lin_best_net = train(epochs, optim, criterion, l1_reg, train_loader, test_loader, lin_net, lin_empty_net, reg_lambda=0.0001)

100%|██████████| 3738/3738 [00:11<00:00, 315.43it/s]
/tmp/ipykernel_47228/1318915428.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x), torch.tensor(y)


Epoch [1/10], Train Loss: 0.2449
Epoch [1/10], Val Loss: 0.1465


100%|██████████| 3738/3738 [00:11<00:00, 323.59it/s]


Epoch [2/10], Train Loss: 0.2339
Epoch [2/10], Val Loss: 0.1489


100%|██████████| 3738/3738 [00:11<00:00, 334.43it/s]


...

Epoch [8/10], Train Loss: 0.2338
Epoch [8/10], Val Loss: 0.1460


100%|██████████| 3738/3738 [00:14<00:00, 256.81it/s]


Epoch [9/10], Train Loss: 0.2339
Epoch [9/10], Val Loss: 0.1467


100%|██████████| 3738/3738 [00:14<00:00, 261.01it/s]


Epoch [10/10], Train Loss: 0.2340
Epoch [10/10], Val Loss: 0.1459


In [161]:
mlp = MLP(384, 512, 384)
empty_mpl = MLP(384, 512, 384)

optim = torch.optim.Adam(lr=lr, params=mlp.parameters())

mlp_last_net, mlp_best_net = train(epochs, optim, criterion, l1_reg, train_loader, test_loader, mlp, empty_mpl, reg_lambda=0.0005)

  0%|          | 0/3738 [00:00<?, ?it/s]

100%|██████████| 3738/3738 [00:24<00:00, 152.85it/s]
/tmp/ipykernel_47228/1318915428.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x), torch.tensor(y)


Epoch [1/10], Train Loss: 0.3130
Epoch [1/10], Val Loss: 0.1274


100%|██████████| 3738/3738 [00:24<00:00, 155.17it/s]


Epoch [2/10], Train Loss: 0.2814
Epoch [2/10], Val Loss: 0.1272


 23%|██▎       | 851/3738 [00:05<00:19, 150.92it/s]


KeyboardInterrupt: 

In [ ]:
test_loss_lin = validate(criterion, test_loader, lin_last_net)
print(f"lin test: {test_loss_lin}")

test_loss_mlp = validate(criterion, test_loader, mlp_last_net)
print(f"lin test: {test_loss_mlp}")
